# $\rho$-VAE:

This is the basic implementaiton of the $rho$-VAE, a simple fix to the vanilla-VAE, which can be extended to other variants as a plug-and-play addition.

In [1]:
import numpy as np
from scipy.linalg import toeplitz
%precision  %.3f

import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
batch_size = 128
epochs = 15
seed = 1
log_interval = 10
z_dim = 5
#database_name = 'mnist'
#database_name = 'cifar10'
database_name = 'fashion'

In [3]:
torch.manual_seed(seed)
device = torch.device("cuda:1")

kwargs = {'num_workers': 1, 'pin_memory': True}
if database_name.lower() == 'mnist':
    database_obj_train = datasets.MNIST('data/mnist', train=True, download=True,
                   transform=transforms.ToTensor())
    database_obj_test = datasets.MNIST('data/mnist', train=False, transform=transforms.ToTensor())

elif database_name.lower() == 'cifar10':
    database_obj_train = datasets.CIFAR10('data/cifar10', train=True, download=True,
                   transform=transforms.ToTensor())
    database_obj_test = datasets.CIFAR10('data/cifar10', train=False, transform=transforms.ToTensor())
    
elif database_name.lower() == 'fashion':
    database_obj_train = datasets.FashionMNIST('data/fashion', train=True, download=True,
                   transform=transforms.ToTensor())
    database_obj_test = datasets.FashionMNIST('data/fashion', train=False, transform=transforms.ToTensor())    
    
    
train_loader = torch.utils.data.DataLoader(database_obj_train,
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(database_obj_test,
    batch_size=batch_size, shuffle=True, **kwargs)
try:
    N_train,image_x,image_y,num_channel = train_loader.dataset.data.shape
    data_dim = image_x * image_y * num_channel
except:
    N_train,image_x,image_y = train_loader.dataset.data.shape
    data_dim = image_x * image_y 
    num_channel = 1

In [4]:
class rho_VAE(nn.Module):
    def __init__(self):
        super(rho_VAE, self).__init__()

        self.fc1 = nn.Linear(data_dim, 400)
        self.fc21 = nn.Linear(400, z_dim)
        self.fc22 = nn.Linear(400, 1)  # rho 
        self.fc23 = nn.Linear(400, 1)  # s
        self.fc3 = nn.Linear(z_dim, 400)
        self.fc4 = nn.Linear(400, data_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), torch.tanh(self.fc22(h1)) , self.fc23(h1)    # -1<rho<1,and s>0

    def reparameterize(self, mu, rho, logs):
        z_q = torch.randn_like(rho).view(-1,1) * torch.sqrt(logs.exp())
        for j in range(1,z_dim):
            addenum = rho * z_q[:,-1].view(-1,1)  + torch.randn_like(rho).view(-1,1) * torch.sqrt(logs.exp())
            z_q = torch.cat(( z_q, addenum ),1)        
        z_q  = z_q + mu  
        
        return z_q

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, rho, logs = self.encode(x.view(-1, data_dim))
        z = self.reparameterize(mu, rho, logs)
        return self.decode(z), mu, rho, logs

In [5]:
model = rho_VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [6]:
def loss_function(recon_x, x, mu, rho, logs):
    
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, data_dim), reduction='sum')
    ##
    KLD = 0.5 * ( torch.sum(mu.pow(2)) + - z_dim * logs - (z_dim - 1) * torch.log(1 - rho**2 + 1e-7) +  z_dim * (logs.exp()-1)  )
    KLD = torch.mean(KLD)

    return BCE + KLD

In [7]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, rho, logs = model(data)
        loss = loss_function(recon_batch, data, mu, rho, logs)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [8]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, rho, logs = model(data)
            test_loss += loss_function(recon_batch, data, mu, rho, logs).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, num_channel, image_x, image_y)[:n]])
                save_image(comparison.cpu(),
                         './samples/rho_VAE/recon_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [9]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, z_dim).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, num_channel, image_x, image_y),'./samples/rho_VAE/sample_' + str(epoch) + '.png')



Train Epoch: 1 [0/60000 (0%)]	Loss: 549.610046
Train Epoch: 1 [1280/60000 (2%)]	Loss: 409.925964
Train Epoch: 1 [2560/60000 (4%)]	Loss: 345.217987
Train Epoch: 1 [3840/60000 (6%)]	Loss: 319.488434
Train Epoch: 1 [5120/60000 (9%)]	Loss: 314.691833
Train Epoch: 1 [6400/60000 (11%)]	Loss: 301.684265
Train Epoch: 1 [7680/60000 (13%)]	Loss: 293.448212
Train Epoch: 1 [8960/60000 (15%)]	Loss: 292.996185
Train Epoch: 1 [10240/60000 (17%)]	Loss: 286.971069
Train Epoch: 1 [11520/60000 (19%)]	Loss: 275.463501
Train Epoch: 1 [12800/60000 (21%)]	Loss: 279.188049
Train Epoch: 1 [14080/60000 (23%)]	Loss: 268.476776
Train Epoch: 1 [15360/60000 (26%)]	Loss: 274.687744
Train Epoch: 1 [16640/60000 (28%)]	Loss: 273.402252
Train Epoch: 1 [17920/60000 (30%)]	Loss: 271.596375
Train Epoch: 1 [19200/60000 (32%)]	Loss: 261.836304
Train Epoch: 1 [20480/60000 (34%)]	Loss: 268.930542
Train Epoch: 1 [21760/60000 (36%)]	Loss: 278.603058
Train Epoch: 1 [23040/60000 (38%)]	Loss: 260.332092
Train Epoch: 1 [24320/60000 

Train Epoch: 4 [17920/60000 (30%)]	Loss: 239.727310
Train Epoch: 4 [19200/60000 (32%)]	Loss: 240.244293
Train Epoch: 4 [20480/60000 (34%)]	Loss: 240.595871
Train Epoch: 4 [21760/60000 (36%)]	Loss: 233.315292
Train Epoch: 4 [23040/60000 (38%)]	Loss: 234.494629
Train Epoch: 4 [24320/60000 (41%)]	Loss: 242.382874
Train Epoch: 4 [25600/60000 (43%)]	Loss: 245.045944
Train Epoch: 4 [26880/60000 (45%)]	Loss: 244.851013
Train Epoch: 4 [28160/60000 (47%)]	Loss: 242.623627
Train Epoch: 4 [29440/60000 (49%)]	Loss: 231.276825
Train Epoch: 4 [30720/60000 (51%)]	Loss: 242.834763
Train Epoch: 4 [32000/60000 (53%)]	Loss: 226.242645
Train Epoch: 4 [33280/60000 (55%)]	Loss: 248.802475
Train Epoch: 4 [34560/60000 (58%)]	Loss: 239.578781
Train Epoch: 4 [35840/60000 (60%)]	Loss: 253.563187
Train Epoch: 4 [37120/60000 (62%)]	Loss: 249.336090
Train Epoch: 4 [38400/60000 (64%)]	Loss: 246.643539
Train Epoch: 4 [39680/60000 (66%)]	Loss: 236.421127
Train Epoch: 4 [40960/60000 (68%)]	Loss: 245.012939
Train Epoch:

Train Epoch: 7 [35840/60000 (60%)]	Loss: 241.365250
Train Epoch: 7 [37120/60000 (62%)]	Loss: 234.996414
Train Epoch: 7 [38400/60000 (64%)]	Loss: 231.764648
Train Epoch: 7 [39680/60000 (66%)]	Loss: 216.605347
Train Epoch: 7 [40960/60000 (68%)]	Loss: 244.121246
Train Epoch: 7 [42240/60000 (70%)]	Loss: 237.955612
Train Epoch: 7 [43520/60000 (72%)]	Loss: 234.087540
Train Epoch: 7 [44800/60000 (75%)]	Loss: 235.375290
Train Epoch: 7 [46080/60000 (77%)]	Loss: 233.774811
Train Epoch: 7 [47360/60000 (79%)]	Loss: 233.294174
Train Epoch: 7 [48640/60000 (81%)]	Loss: 240.086594
Train Epoch: 7 [49920/60000 (83%)]	Loss: 236.476898
Train Epoch: 7 [51200/60000 (85%)]	Loss: 236.249542
Train Epoch: 7 [52480/60000 (87%)]	Loss: 238.324158
Train Epoch: 7 [53760/60000 (90%)]	Loss: 228.666199
Train Epoch: 7 [55040/60000 (92%)]	Loss: 223.979614
Train Epoch: 7 [56320/60000 (94%)]	Loss: 224.463715
Train Epoch: 7 [57600/60000 (96%)]	Loss: 232.417053
Train Epoch: 7 [58880/60000 (98%)]	Loss: 223.857697
====> Epoch:

Train Epoch: 10 [52480/60000 (87%)]	Loss: 239.544601
Train Epoch: 10 [53760/60000 (90%)]	Loss: 228.607544
Train Epoch: 10 [55040/60000 (92%)]	Loss: 233.496353
Train Epoch: 10 [56320/60000 (94%)]	Loss: 243.450562
Train Epoch: 10 [57600/60000 (96%)]	Loss: 222.740265
Train Epoch: 10 [58880/60000 (98%)]	Loss: 224.990341
====> Epoch: 10 Average loss: 232.0629
====> Test set loss: 233.9971
Train Epoch: 11 [0/60000 (0%)]	Loss: 234.344986
Train Epoch: 11 [1280/60000 (2%)]	Loss: 225.215424
Train Epoch: 11 [2560/60000 (4%)]	Loss: 240.003220
Train Epoch: 11 [3840/60000 (6%)]	Loss: 229.920502
Train Epoch: 11 [5120/60000 (9%)]	Loss: 236.543503
Train Epoch: 11 [6400/60000 (11%)]	Loss: 222.134094
Train Epoch: 11 [7680/60000 (13%)]	Loss: 227.244324
Train Epoch: 11 [8960/60000 (15%)]	Loss: 227.170639
Train Epoch: 11 [10240/60000 (17%)]	Loss: 225.409622
Train Epoch: 11 [11520/60000 (19%)]	Loss: 231.928635
Train Epoch: 11 [12800/60000 (21%)]	Loss: 226.657761
Train Epoch: 11 [14080/60000 (23%)]	Loss: 222.

Train Epoch: 14 [5120/60000 (9%)]	Loss: 239.930725
Train Epoch: 14 [6400/60000 (11%)]	Loss: 219.601746
Train Epoch: 14 [7680/60000 (13%)]	Loss: 240.580444
Train Epoch: 14 [8960/60000 (15%)]	Loss: 238.970703
Train Epoch: 14 [10240/60000 (17%)]	Loss: 229.269058
Train Epoch: 14 [11520/60000 (19%)]	Loss: 229.061172
Train Epoch: 14 [12800/60000 (21%)]	Loss: 237.237427
Train Epoch: 14 [14080/60000 (23%)]	Loss: 224.794098
Train Epoch: 14 [15360/60000 (26%)]	Loss: 226.153168
Train Epoch: 14 [16640/60000 (28%)]	Loss: 229.795914
Train Epoch: 14 [17920/60000 (30%)]	Loss: 231.021896
Train Epoch: 14 [19200/60000 (32%)]	Loss: 236.554733
Train Epoch: 14 [20480/60000 (34%)]	Loss: 225.471710
Train Epoch: 14 [21760/60000 (36%)]	Loss: 227.749756
Train Epoch: 14 [23040/60000 (38%)]	Loss: 225.867462
Train Epoch: 14 [24320/60000 (41%)]	Loss: 225.082657
Train Epoch: 14 [25600/60000 (43%)]	Loss: 214.534607
Train Epoch: 14 [26880/60000 (45%)]	Loss: 219.408035
Train Epoch: 14 [28160/60000 (47%)]	Loss: 231.24911